## Overview

Link to my fork: https://github.com/JaymanR/llm_engineering

For the 3-way chatbot, I have used gpt 4.1 mini, gemini 2.5 flash lite and llama3.2 via ollama.

In [ ]:
!ollama pull llama3.2

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from ollama import chat

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set.")

In [ ]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
MODEL_GPT = "gpt-4.1-mini"
MODEL_GEMINI = "gemini-2.5-flash-lite"
MODEL_LLAMA = "llama3.2"

BOT1 = "Alex"
BOT2 = "Blake"
BOT3 = "Cosmo"

CHATBOTS = [BOT1, BOT2, BOT3]

In [ ]:
def get_other_bots(current_bot):
    return [bot for bot in CHATBOTS if bot != current_bot]

def get_user_prompt(current_bot, conversation):
    other_bots = get_other_bots(current_bot)
    return f"""
    You are {current_bot}, in a conversation with {other_bots[0]} and {other_bots[1]}.
    The conversation so far is as follows:
    {conversation}
    Now with this, respond with what you would like to say next, as {current_bot}.
    """

In [ ]:
gpt_system_prompt = f"""
You are {BOT1}, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with {BOT3} and {BOT2}. Ensure your response starts with, {BOT1}: 
"""

gemini_system_prompt = f"""
You are {BOT2}, a chatbot who lives in your own bubble. You are in a conversation with {BOT1} and {BOT3}
You often start on topic, but eventually drift on to ramble about things out of scope without realizing during the conversation. 
Ensure your response starts with, {BOT2}: 
"""

# I tried asking llama3.2 to start with Bot_Name: , but sometimes it wouldn't start with it.
llama_system_prompt = f"""
You are {BOT3}, a very polite, courteous chatbot. You are in a conversation with {BOT1} and {BOT2}
You try to agree with everything the other person says, or find common ground. 
If the other person is argumentative, you try to calm them down and keep chatting.
Please directly respond as if you are talking to them both live.
"""

In [ ]:
gpt_messages = [f"{BOT1}: Hi, I guess..."]
gemini_messages = [f"{BOT2}: Hello!"]
ollama_messages = [f"{BOT3}: Hello, How are you doing?"]

In [ ]:
def build_conversation():
    conversation = ""
    max_len = max(len(gpt_messages), len(ollama_messages), len(gemini_messages))
    for i in range(max_len):
        if i < len(gpt_messages):
            conversation += gpt_messages[i] + "\n\n"
        if i < len(gemini_messages):
            conversation += gemini_messages[i] + "\n\n"
        if i < len(ollama_messages):
            conversation += ollama_messages[i] + "\n\n"
    return conversation

In [ ]:
def get_messages(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
        ]

In [ ]:
def call_gpt():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=get_messages(gpt_system_prompt, get_user_prompt(BOT1, build_conversation())),
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [ ]:
def call_gpt():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=get_messages(gpt_system_prompt, get_user_prompt(BOT1, build_conversation())),
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [ ]:
def call_gemini():
    stream = gemini.chat.completions.create(
        model=MODEL_GEMINI,
        messages=get_messages(gemini_system_prompt, get_user_prompt(BOT2, build_conversation())),
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [ ]:
def call_ollama():
    stream = chat(
        model=MODEL_LLAMA,
        messages=get_messages(llama_system_prompt, get_user_prompt(BOT3, build_conversation())),
        stream=True
    )
    response = f"{BOT3}: "
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.message.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [ ]:
gpt_messages = [f"{BOT1}: Hi, I guess..."]
gemini_messages = [f"{BOT2}: Hello!"]
ollama_messages = [f"{BOT3}: Hello, How are you doing?"]

display(Markdown(build_conversation()))

call_gpt()
call_gemini()
call_ollama()

for i in range(2):
    gpt_next = call_gpt()
    gpt_messages.append(gpt_next)

    gemini_next = call_gemini()
    gemini_messages.append(gemini_next)

    ollama_next = call_ollama()
    ollama_messages.append(ollama_next)